In [35]:
# tokenization, stemming, stop word removal
import xlrd, re

path = ("Tweets.xls") 
spreadSheet = xlrd.open_workbook(path)

testData = []
tweetsData = []
sentiment = [] 
targetSentiment = ["positive", "negative","neutral"]

import random
sheet1 = spreadSheet.sheet_by_index(1) 
#Training Data ------
for x in range(1, 420):
    i = random.randint(1,420)
#     print("rndom:" , i)
    arrStr = re.sub('[!@#\"$]', "", sheet1.cell_value(i, 10)) 
    arrStr = re.sub(r"\//\S+", "", arrStr)
    tweetsData.append(arrStr) 
    sentiment.append(sheet1.cell_value(i, 1))
     
# print(tweetsData)
# print(testData)

#Test Data -------
testDict = []
testSentiment = []
sheet0 = spreadSheet.sheet_by_index(0) 

for count in range(1, 20):
    x = random.randint(10,200)
   
    arrStr = re.sub('[!@#\"$]', "", sheet0.cell_value(x, 10))   
    testDict.append(arrStr)
    testSentiment.append(sheet0.cell_value(x, 1))
    
# print("TEST", testDict)
print("SENTIMENT TEST", testSentiment)


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

countVector = CountVectorizer()
vocab= countVector.fit(tweetsData)
# print("\nVocabulary: " + str(countVector.vocabulary_) + '\n')
# print("Features: " + str(countVector.get_feature_names()) + '\n') 

termCountMatrix = countVector.transform(tweetsData)
print("SHAPE: "+ str(termCountMatrix.shape)) #3 documents and 7 unique terms/vocab
# print("COUNT MATRIX: \n"+ str(termCountMatrix.toarray())) 

#terms given tfIdf
tfIdfTransformer = TfidfTransformer()
tfIdfTransformer.fit(termCountMatrix)#sorted in the order of Features array
# print("TFIDF:" + str(tfIdfTransformer.idf_))

tdIdfMatrix = tfIdfTransformer.transform(termCountMatrix)
# print("TFIDF:" + str(tdIdfMatrix.toarray()))

model = MultinomialNB().fit(tdIdfMatrix,sentiment)
# model = make_pipeline(TfidfVectorizer(), MultinomialNB())
# model.fit(trainDict.values(), trainDict.keys())
# labels = model.predict(testDict.values())

newVectorMatrix = countVector.transform(testDict)
newTFMatrix =  tfIdfTransformer.transform(newVectorMatrix)
predicted = model.predict(newTFMatrix)

print("PREDICT",predicted)


from sklearn import metrics

accuracy = metrics.accuracy_score(testSentiment,predicted)
print("accuracy:", accuracy)
print(metrics.classification_report(testSentiment, predicted, targetSentiment))
metrics.confusion_matrix(testSentiment,predicted)



SENTIMENT TEST ['neutral', 'neutral', 'negative', 'negative', 'neutral', 'positive', 'negative', 'negative', 'negative', 'neutral', 'negative', 'negative', 'positive', 'neutral', 'negative', 'neutral', 'negative', 'neutral', 'positive']
SHAPE: (419, 1218)
PREDICT ['positive' 'positive' 'positive' 'positive' 'positive' 'positive'
 'positive' 'positive' 'positive' 'positive' 'positive' 'positive'
 'positive' 'positive' 'positive' 'neutral' 'positive' 'positive'
 'positive']
accuracy: 0.21052631578947367
              precision    recall  f1-score   support

    positive       0.17      1.00      0.29         3
    negative       0.00      0.00      0.00         9
     neutral       1.00      0.14      0.25         7

   micro avg       0.21      0.21      0.21        19
   macro avg       0.39      0.38      0.18        19
weighted avg       0.39      0.21      0.14        19



/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


array([[0, 0, 9],
       [0, 1, 6],
       [0, 0, 3]])